In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import warnings

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.layers import Dense, Input, InputLayer, LeakyReLU, Flatten

In [3]:
warnings.filterwarnings('ignore')
pd.set_option('display.float_format','{:.3f}'.format)
pd.set_option('display.max_columns',None)

In [4]:
churn_df = pd.read_csv('churn_cleaned2.csv')

In [5]:
churn_df.head()

,gender,age,no_of_days_subscribed,multi_screen,mail_subscribed,weekly_mins_watched,minimum_daily_mins,weekly_max_night_mins,videos_watched,customer_support_calls,churn
0,0,-0.128,-0.950,0,0,-1.513,0.719,-0.943,-1.977,-0.239,0
1,0,0.188,1.239,0,0,0.301,-0.897,-0.687,-0.513,0.551,0
2,0,2.079,0.660,0,0,-2.271,0.611,-0.482,-1.977,1.975,1
3,0,-1.817,0.786,0,1,0.635,-0.251,0.081,-0.016,1.130,0
4,0,0.286,2.296,0,0,-0.337,0.252,-0.892,1.075,-0.239,0


In [6]:
churn_df['churn'].value_counts()

0    1738
1     262
Name: churn, dtype: int64

In [7]:
churn_df['churn'].value_counts(normalize=True)

0   0.869
1   0.131
Name: churn, dtype: float64

In [8]:
1738/262

6.633587786259542

In [9]:
X = churn_df.drop(columns=['churn'])
y = churn_df['churn']

In [10]:
smote = SMOTE(random_state=47)

X_new, y_new = smote.fit_resample(X,y)

In [11]:
y_new.value_counts()

0    1738
1    1738
Name: churn, dtype: int64

In [12]:
X_new.shape, y_new.shape

((3476, 10), (3476,))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y_new,test_size=0.2,random_state=32)

In [14]:
X_train.shape, y_train.shape

((2780, 10), (2780,))

In [15]:
X_test.shape, y_test.shape

((696, 10), (696,))

In [16]:
model = Sequential()

model.add(Flatten(input_shape=(10,)))
#model.add(Dense(units=64, activation='relu', input_shape=(10,)))

model.add(Dense(128,activation='relu'))

model.add(Dense(128,activation='tanh'))

#model.add(Dense(128,LeakyReLU(0.1)))

model.add(Dense(1,activation='sigmoid'))

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           1,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,049 (70.50 KB)

 Trainable params: 18,049 (70.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')])

In [19]:
history = model.fit(X_train, y_train, batch_size=64, epochs=40, validation_split=0.15, verbose=True)

Epoch 1/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6506 - loss: 0.6224 - val_accuracy: 0.7290 - val_loss: 0.5359
Epoch 2/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7617 - loss: 0.5020 - val_accuracy: 0.7842 - val_loss: 0.4799
Epoch 3/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7845 - loss: 0.4687 - val_accuracy: 0.7986 - val_loss: 0.4645
Epoch 4/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7845 - loss: 0.4509 - val_accuracy: 0.8249 - val_loss: 0.4517
Epoch 5/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8137 - loss: 0.4281 - val_accuracy: 0.7986 - val_loss: 0.4482
Epoch 6/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8174 - loss: 0.4112 - val_accuracy: 0.7914 - val_loss: 0.4447
Epoch 7/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8076 - loss: 0.4073 - val_accuracy: 0.8153 - val_loss: 0.4366
Epoch 8/40
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8155 - loss: 0.4054 - val_accuracy: 0.8129 - val_loss

- Model Evaluation

In [20]:
test_loss, test_acc = model.evaluate(X_test,y_test,batch_size=64,verbose=True)
test_acc

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8256 - loss: 0.4524 


0.8477011322975159

- Model Save

In [21]:
model.save('ANN_Churn_Classifier.keras')

- Save the weights

In [22]:
model.save_weights('ANN_Churn_classifier.weights.h5')   # ".weights.h5" extension